### Load Integral

In [194]:
Integral = nil
package.loaded['Integral-jit'] = nil

torch.setdefaulttensortype('torch.FloatTensor')

require 'Integral-jit'

### Visual correctness check

In [195]:
cv = require 'cv'
require 'cv.highgui'
require 'image'

res = Integral(12, 512, 512):forward(image.lena():mean(1):squeeze())

cv.namedWindow{'w'}
cv.imshow{'w', image.lena():mean(1):squeeze()}
cv.waitKey{0}
cv.imshow{'w', image.lena():mean(1):squeeze()}
cv.waitKey{0}
cv.imshow{'w', image.lena():mean(1):squeeze()}
cv.waitKey{0}

for i = 1,res:size(1) do
    cv.imshow{'w', cv.applyColorMap{
            (res[i] / res[i]:max() * 255):byte(), nil, cv.COLORMAP_HOT}}
    cv.waitKey{0}
end

cv.destroyAllWindows{}

res = nil
collectgarbage()

### Measure execution times

Input: 1024x768 image

* **Forward** experiment: compute `16x1024x768` feature maps 4 times, repeat 6 times
* **Backward** experiment: do a backprop step (from the previous state) 4 times, repeat 6 times

#### Forward

LuaJIT: $0.487 \pm 0.087$ seconds

#### Backward

LuaJIT: $2.660 \pm 0.252$ seconds

In [196]:
h = 768
w = 1024
nMaps = 16

In [197]:
int = Integral(16, 768, 1024)
params, gradParams = int:getParameters()

#### Forward experiment

In [207]:
img = torch.rand(h, w)
local repeats = 4
local timeRepeats = 6

local times = torch.Tensor(timeRepeats)

for timeRepeat = 1,timeRepeats do

    local timer = torch.Timer()

    for _ = 1,repeats do
        int:forward(img)
    end

    timer:stop()

    times[timeRepeat] = timer:time().real
    
    collectgarbage()
    
end

print(times:mean() .. ' +/- ' .. 2.1 * times:std() .. ' seconds')

0.48705518245697 +/- 0.086716275472322 seconds	


#### Backward experiment

In [208]:
img = torch.rand(h, w)
local repeats = 4
local timeRepeats = 6

local times = torch.Tensor(timeRepeats)

int:forward(img)
int:backward(img, int.output)

for timeRepeat = 1,timeRepeats do

    local timer = torch.Timer()

    for _ = 1,repeats do
        int:backward(img, int.output)
    end

    timer:stop()

    times[timeRepeat] = timer:time().real
    
    collectgarbage()
    
end

print(times:mean() .. ' +/- ' .. 2.1 * times:std() .. ' seconds')

2.6598696708679 +/- 0.25184703056895 seconds	
